In [27]:
import pandas as pd
import numpy as np
import os
from os.path import join as oj
from pose_analysis_base import *
import matplotlib.pyplot as plt

In [28]:
root_folder = r'/Users/yang/Documents/Wilbrecht_Lab/data/processed_tracks'

posemat = PoseMat(root_folder)
combined_df = posemat.get_combined_df()

Loading combined DataFrame...


In [29]:
print(combined_df.shape)
combined_df.head()

(1726631, 31)


,Head x,Head y,Neck x,Neck y,Torso x,Torso y,Tailhead x,Tailhead y,warped Head x,warped Head y,...,lapIndex,trial,decision,Head_vx,Head_vy,Head_v,angular_velocity,dPhi,animal,session
0,188.623566,28.475878,191.675903,23.529375,195.564896,11.922485,199.491653,6.846012,579.176095,299.356675,...,0,4,NaN,33.297947,-492.902488,494.025926,-3.850601,0.007309,RRM028,Day141
1,188.060455,35.737679,189.072708,28.708460,192.960205,18.967358,196.653687,7.442016,580.286027,282.926592,...,0,4,NaN,29.372719,-487.785702,488.669262,-2.622566,0.006488,RRM028,Day141
2,187.637146,43.289207,188.460831,36.384815,192.207230,24.570004,195.903427,12.627603,581.134276,266.837628,...,0,4,NaN,26.170808,-479.931514,480.644535,-0.853533,0.014482,RRM028,Day141
3,187.621353,48.790691,188.307617,43.796303,191.519562,32.192448,195.331573,20.492044,582.030747,250.931157,...,0,4,NaN,44.096649,-493.561166,495.527133,0.203085,0.045977,RRM028,Day141
4,187.166351,56.842167,187.897903,51.815838,191.288147,39.605297,195.342529,27.651142,584.074053,233.933550,...,0,4,NaN,77.330604,-524.327621,529.999506,0.472210,0.025462,RRM028,Day141


In [30]:
def compare_decisions(df):
    current_trial_num = np.nan
    current_bonsai_decision = np.nan
    current_decision = np.nan
    bonsai_decision = np.nan
    decision = np.nan
    
    bonsai_decision_list = []
    decision_list = []
    animal_list = []
    session_list = []
    trial_list = []

    for index, row in df.iterrows():
        bonsai_decision = row['label']
        decision = row['decision']
        trial_num = row['trial']
        animal = row['animal']
        session = row['session']
        
        if trial_num != current_trial_num: # End of current trial and start of a new trial 
            if current_bonsai_decision == 'collection':
                current_bonsai_decision = 'ACC'
            if current_decision != current_bonsai_decision:
                bonsai_decision_list.append(current_bonsai_decision)
                decision_list.append(current_decision)
                animal_list.append(animal)
                session_list.append(session)
                trial_list.append(current_trial_num)

            current_bonsai_decision = np.nan    
            current_decision = np.nan
            current_trial_num = trial_num 
              
        if not pd.isna(decision):
            current_decision = decision    
        if not pd.isna(bonsai_decision):
            current_bonsai_decision = bonsai_decision
    
    result_df = pd.DataFrame({
        'bonsai decision': bonsai_decision_list,
        'decision': decision_list,
        'animal': animal_list,
        'session': session_list,
        'trial': trial_list
    })
    
    return result_df

In [31]:
different_trials = compare_decisions(combined_df)

In [32]:
#different_trials.to_csv(oj(root_folder, 'different_trials.csv'), index=False)
print(different_trials.shape)
different_trials.head()

(2250, 5)


,bonsai decision,decision,animal,session,trial
0,NaN,NaN,RRM028,Day141,NaN
1,quit,REJ,RRM028,Day141,66.0
2,quit,REJ,RRM028,Day141,85.0
3,quit,REJ,RRM028,Day141,97.0
4,quit,REJ,RRM028,Day141,107.0


In [33]:
ACC_REJ_trials = different_trials[
    ((different_trials['bonsai decision'] == 'ACC') & (different_trials['decision'] == 'REJ')) |
    ((different_trials['bonsai decision'] == 'REJ') & (different_trials['decision'] == 'ACC'))
]

quit_REJ_trials = different_trials[
    (different_trials['bonsai decision'] == 'quit') & (different_trials['decision'] == 'REJ')
]

print(len(ACC_REJ_trials))
print(len(quit_REJ_trials))

9
1968


In [34]:
a = different_trials[
    different_trials['bonsai decision'] == np.nan
]
a

,bonsai decision,decision,animal,session,trial


In [35]:
# Get unique combinations of decisions that exist in the DataFrame
existing_combinations = different_trials[['bonsai decision', 'decision']].drop_duplicates()

# Convert to list of tuples
existing_combinations_list = [tuple(x) for x in existing_combinations.to_records(index=False)]

# Display the combinations
print(existing_combinations_list)

[(nan, nan), ('quit', 'REJ'), ('T_Entry', 'REJ'), ('T_Entry', 'quit'), ('REJ', 'quit'), ('ACC', 'REJ'), ('ACC', 'T_Entry'), ('quit', 'ACC'), ('T_Entry', 'ACC'), ('0tone', 'quit'), ('ACC', 'quit'), ('80tone', 'REJ'), ('T_Entry', nan)]
